In [1]:
# Import modules
import numpy as np

# Import functions
from local_global_matrices import *
from assembly_K_matrices import *

%load_ext autoreload
%autoreload 2

In [2]:
# Import data
d = np.fromfile('data/d.dat')
fP = np.fromfile('data/fP.dat')
fr = np.fromfile('data/fr.dat')
Ks = np.genfromtxt('data/localK.dat')
solution = np.fromfile('data/solution.dat')

In [3]:
# Initial data
# Number of subdomains
Nsub_x = 4
Nsub_y = 3

# Number of remaining nodes in each subdomain
Nr_x = 4
Nr_y = 3

# Local remaining and primal indices
rs = np.array([1, 2, 4, 5, 6, 7, 9, 10])
qs = np.array([0, 3, 8, 11])

In [4]:
# Transformation matrices A
# Primal nodes local-global transformation matrices
APq = create_APq_matrices(Nsub_x, Nsub_y)

# Remaining nodes local-global transformation matrices
ARr = create_ARr_matrices(Nsub_x, Nsub_y, Nr_x, Nr_y)

In [5]:
# Stiffness matrices K
KRR = assembly_KRR_matrix(Ks, ARr, rs)
KPP = assembly_KPP_matrix(Ks, APq, qs)
KPR = assembly_KPR_matrix(Ks, APq, ARr, qs, rs)
KRP = KPR.T